<a href="https://colab.research.google.com/github/HamiltonLROliveira/ciencia_dados_puc_rj/blob/main/anal_expl_viagens_gov_federal_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Importação de pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from datetime import datetime

In [ ]:
# Carrega arquivo csv usando Pandas usando uma URL

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/HamiltonLROliveira/ciencia_dados_puc_rj/main/2023_Passagem.zip"

# Lê o arquivo utilizando as colunas informadas
df_passagens = pd.read_csv(url, sep=';', encoding='latin1' )

In [ ]:
df_passagens

In [ ]:
df_passagens.info()

In [ ]:
df_passagens.head(10)

In [ ]:
"""
Tratamento em todas as colunas para colocar o valor None quando o valor for 'Não informado' ou NaN
"""
for c in df_passagens.columns:
    df_passagens[c] = df_passagens[c].apply(lambda x: x if x != 'Sem Informação' and pd.notnull(x) else None)

In [ ]:
"""
A conversão pode ser feita utilizando o método apply da série que deseja converter.
"""
def converter_numero(valor):
    try:
        return float(valor.replace('.', '').replace(',', '.'))
    except:
        return np.nan

In [ ]:
df_passagens_1 = df_passagens.copy()
df_passagens_1['Valor da passagem'] = \
    df_passagens_1['Valor da passagem'].apply(converter_numero)
df_passagens_1['Taxa de serviço'] =  \
    df_passagens_1['Taxa de serviço'].apply(converter_numero)

In [ ]:
df_passagens_1[['Valor da passagem', 'Taxa de serviço']].head()
df_passagens_1.info()

In [ ]:
"""
Padronização das strings é interessante para que a ausência
ou a presença de acentos não interfira em agrupamentos.
"""

from texto import TratamentoTexto
def tratar_texto(valor):
    v = valor

    # somente realiza o tratamento se o texto não for nulo
    if v:
        # remove acentuação
        v = TratamentoTexto.remover_acentuacao(v)
        # converte para letras maiúsculas
        v = v.upper()

    return v

In [ ]:
colunas = ['Meio de transporte',
       'País - Origem ida', 'UF - Origem ida', 'Cidade - Origem ida',
       'País - Destino ida', 'UF - Destino ida', 'Cidade - Destino ida',
       'País - Origem volta', 'UF - Origem volta', 'Cidade - Origem volta',
       'Pais - Destino volta', 'UF - Destino volta', 'Cidade - Destino volta']

# padronize as colunas da lista acima no dataframe df_passagens_1
for c in colunas:
    df_passagens_1[c] = df_passagens_1[c].apply(tratar_texto)

In [ ]:
df_passagens_1.head()

In [ ]:
colunas_gb = ['País - Origem ida', 'UF - Origem ida', 'Cidade - Origem ida',
       'País - Destino ida', 'UF - Destino ida', 'Cidade - Destino ida']

colunas_selecao = ['País - Origem ida', 'UF - Origem ida',
                   'Cidade - Origem ida',
       'País - Destino ida', 'UF - Destino ida',
                   'Cidade - Destino ida', 'Valor da passagem']

df_passagens_1[colunas_selecao]\
    .groupby(colunas_gb, as_index=False)\
    .agg(['count', 'mean', 'median'])\
    .sort_values(('Valor da passagem', 'count'), ascending=False)\
    .head(10)

In [ ]:
df_passagens_1['Identificador do processo de viagem']\
    .value_counts()\
    .value_counts()

In [ ]:
plt.rcParams['figure.figsize'] = (16.5,6)
df_passagens_1['Identificador do processo de viagem']\
    .value_counts()\
    .value_counts().plot.bar()
plt.xlabel('Quantidade de Trechos por ID de Viagem')
plt.ylabel('Quantidade de Registros')
#plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
grouped = df_passagens_1.groupby('Identificador do processo de viagem').size().to_frame()

In [ ]:
grouped[grouped[0].sort_values(ascending=True) ==5]

In [ ]:
'''
Exemplo de uma viagem com 2 trechos
'''
df_passagens_1[ df_passagens_1['Identificador do processo de viagem'] == 18352553]

In [ ]:
'''
Exemplo de uma viagem com 5 trechos
'''
df_passagens_1[ df_passagens_1['Identificador do processo de viagem'] == 18536121]

In [ ]:
"""
Vamos criar uma coluna que identifica se o trecho faz parte de uma viagem internacional ou não
"""

# vamos marcar trechos que são de origem ou destino internacional
df_passagens_1['Trecho Internacional'] = \
    (df_passagens_1['País - Origem ida'] != 'BRASIL') | (df_passagens_1['País - Destino ida'] != 'BRASIL') \
    | ( df_passagens_1['País - Origem volta'].notnull() & ((df_passagens_1['País - Origem volta'] != 'BRASIL') | (df_passagens_1['Pais - Destino volta'] != 'BRASIL'))) \

df_passagens_1[df_passagens_1['Trecho Internacional']].head()

In [ ]:
# quais viagens possuem ao menos um trecho internacional
df_viagens_internacionais = \
    df_passagens_1[['Identificador do processo de viagem','Trecho Internacional']]\
    .groupby('Identificador do processo de viagem', as_index=False)\
    .max()

df_viagens_internacionais.head()

In [ ]:
# quais processos de viagem possuem trechos internacionais
df_viagens_internacionais = df_viagens_internacionais.rename(columns={'Trecho Internacional': 'Viagem Internacional'})
df_viagens_internacionais.head()

In [ ]:
df_passagens_2 = pd.merge(
        df_passagens_1, df_viagens_internacionais, how='left',
                          left_on='Identificador do processo de viagem', right_on='Identificador do processo de viagem')
df_passagens_2.head()

In [ ]:
df_passagens_2[(df_passagens_2['Trecho Internacional'] == False) & (df_passagens_2['Viagem Internacional'] == True)]

In [ ]:
df_passagens_2[df_passagens_2['Identificador do processo de viagem'] == 19430498]

In [ ]:
df_passagens_g = df_passagens_2[df_passagens_2['Valor da passagem'] > 0.001]
plt.rcParams['figure.figsize'] = 16,10
ax = sns.boxplot(y=df_passagens_g['Valor da passagem'], x=df_passagens_g['Trecho Internacional'].apply(lambda x: 'Internacional' if x else 'Nacional'))
ax.set_yscale('log')
plt.grid(True)
plt.show()

In [ ]:
# Lê o arquivo
dataset_pagamento = pd.read_csv("../2023_Pagamento.csv", encoding='latin1', on_bad_lines = 'warn', sep=';')

In [ ]:
# Mostra as dimensões do dataset
dataset_pagamento.shape

In [ ]:
dataset_pagamento.head(10)

In [ ]:
dataset_pagamento.tail(10)

In [ ]:
# Mostra as informações do dataset
dataset_pagamento.info()

In [ ]:
dataset_pagamento['Identificador do processo de viagem'] = dataset_pagamento['Identificador do processo de viagem'].astype(str)

In [ ]:
def converter_numero(valor):
    try:
        return float(valor.replace('.', '').replace(',', '.'))
    except:
        return np.nan

In [ ]:
dataset_pagamento['Valor'] = dataset_pagamento['Valor'].apply(converter_numero)

In [ ]:
dataset_pagamento.info()

In [ ]:
# Mostra as dimensões do dataset
dataset_passagem.shape

In [ ]:
dataset_passagem.head(10)

In [ ]:
dataset_passagem[ dataset_passagem['Identificador do processo de viagem'] == 18345549 ]

In [ ]:
dataset_trecho[ dataset_trecho['Identificador do processo de viagem '] == 18345549 ]

In [ ]:
dataset_trecho.columns

In [ ]:
dataset_passagem.tail(10)

In [ ]:
# Mostra as informações do dataset
dataset_passagem.info()

In [ ]:
dataset_passagem['Identificador do processo de viagem'] = dataset_passagem['Identificador do processo de viagem'].astype(str)

In [ ]:
dataset_passagem['Valor da passagem'] = dataset_passagem['Valor da passagem'].apply(converter_numero)

In [ ]:
def converter_data(dt):
    try:
        return datetime.strptime(dt, '%d/%m/%Y').date()
    except:
        return np.nan

In [ ]:
dataset_passagem['Data da emissão/compra'] = dataset_passagem['Data da emissão/compra'].apply(converter_data)

In [ ]:
dataset_passagem.info()

In [ ]:
dataset_trecho = pd.read_csv("../2023_Trecho.csv", encoding='latin1', on_bad_lines = 'warn', sep=';')

In [ ]:
# Mostra as dimensões do dataset
dataset_trecho.shape

In [ ]:
dataset_trecho.head(10)

In [ ]:
dataset_trecho.tail(10)

In [ ]:
# Mostra as informações do dataset
dataset_trecho.info()

In [ ]:
dataset_trecho['Identificador do processo de viagem '] = dataset_trecho['Identificador do processo de viagem '].astype(str)

In [ ]:
dataset_trecho.info()

In [ ]:
dataset_trecho.head()

In [ ]:
dataset_viagem = pd.read_csv("../2023_Viagem.csv", encoding='latin1', on_bad_lines = 'warn', sep=';')

In [ ]:
# Mostra as dimensões do dataset
dataset_viagem.shape

In [ ]:
dataset_viagem.head(10)

In [ ]:
dataset_viagem.tail(10)

In [ ]:
# Mostra as informações do dataset
dataset_viagem.info()